# Benchmarking word embeddings

- CUE: Normed Word Cue
- TARGET: Response to Normed Word
- #G: The number of participants serving in the group norming the word, 
- #P: The number of participants producing a particular response.
- FSG: Forward Cue-to-Target Strength  
    - dividing #P by #G which gives the proportion of subjects in the group who produce a particular target in the presence of the cue word.
- BSG: Backward Target-to-Cue Strength
- OSG: Overlapping strength. 
    - Two words comprising a particular pair may also have associates in common, what have sometimes been called overlapping, convergent or shared associates. The cue word and the target word may produce some of the same words as associates. For example, both ABILITY and CAPABILITY produce the same 6 words as associates, including able, strength, talent, potential, capacity, and knowledge. The overlap strength for this pair is calculated as shown in Table 2. From this example, it should be clear that OSG is calculated like MSG in that the strengths of the individual connections are cross multiplied and then summed.

### University of South Florida Free Association Norms
- see more at http://w3.usf.edu/FreeAssociation/

### SimLex-999
- see more at https://fh295.github.io//simlex.html
- paper: Faruqui, M., Tsvetkov, Y., Rastogi, P., & Dyer, C. (2016). Problems With Evaluation of Word Embeddings Using Word Similarity Tasks. 30–35. https://doi.org/10.18653/v1/w16-2506

In [ ]:
%load_ext lab_black
import pandas as pd

### List all existing overlapped words in BERT and LSA-TASA

In [ ]:
import numpy as np
from random import sample
# Select overlapping set in TASA and BERT
bert = pd.read_csv('input/df_train_bert.csv')
tasa = pd.read_csv('input/df_train_tasa.csv')

bert_vec = np.load('input/y_train_bert.npz')['data']
tasa_vec = np.load('input/y_train_tasa.npz')['data']

selwords = list(tasa.word[tasa.word.isin(bert.word)].str.lower())
print('Some samples: {}'.format(sample(selwords, 10)))

### Select and clean Free Association Norm

In [ ]:
fan = pd.read_csv('benchmark/free_asso/fan_manual.csv')
sel_df = fan.loc[:, ['CUE', ' TARGET', ' OSG']]
sel_df.dropna(inplace=True)
sel_df.columns = ['cue', 'target', 'sim']
sel_df['cue'] = sel_df.cue.str.strip().str.lower()
sel_df['target'] = sel_df.target.str.strip().str.lower()

### Select and clean SimLex-999

In [ ]:
sl999 = pd.read_csv('benchmark/simlex_999/SimLex-999.txt', delimiter='\t')
sl999 = sl999.loc[:, ['word1', 'word2', 'SimLex999']]


### Create test case by overlapping items with two embeddings

In [ ]:
sel_df['cue_chk'] = sel_df.cue.isin(selwords)
sel_df['tar_chk'] = sel_df.target.isin(selwords)
sel_df['double_chk'] = sel_df.cue_chk & sel_df.tar_chk
testcase_sem = sel_df.loc[sel_df.double_chk,
                          ['cue', 'target', 'osg']].reset_index(drop=True)

In [ ]:
from tensorflow.keras.losses import cosine_similarity as cosdis


def w2vec(word, words_df, words_vec):
    if len(words_df) != len(words_vec):
        print('CAUTION: WORDS DF AND WORDS VECTOR DONT MATCH IN LENGTH!!')
    if word in list(words_df):
        vec_id = words_df.index[words_df == word]
        return words_vec[vec_id]
    else:
        print('not on list')

In [ ]:
w2vec('lamp', bert.word, bert_vec)

### Run throught test case

In [ ]:
def cal_similarity(w1, w2, emb):
    if emb == 'bert':
        words = bert.word
        vecs = bert_vec
    elif emb == 'tasa':
        words = tasa.word
        vecs = tasa_vec
    else:
        print('Use option emb = "tasa" or "bert"')

    return -cosdis(w2vec(w1, words, vecs), w2vec(w2, words, vecs)).numpy()

In [ ]:
from tqdm import tqdm

bert_similarity = []
tasa_similarity = []

for i in tqdm(range(len(testcase_sem))):

    bert_similarity.append(
        cal_similarity(
            testcase_sem.cue.str.lower()[i],
            testcase_sem.target.str.lower()[i], 'bert'
        )[0]
    )

    tasa_similarity.append(
        cal_similarity(
            testcase_sem.cue.str.lower()[i],
            testcase_sem.target.str.lower()[i], 'tasa'
        )[0]
    )
    
testcase_sem['tasa_similarity'] = tasa_similarity
testcase_sem['bert_similarity'] = bert_similarity

testcase_sem.to_csv('testcase_sim.csv')

# Calculate correlations / cosine similarity

In [ ]:
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import cosine

print(pearsonr(testcase_sem.tasa_similarity, testcase_sem.bert_similarity)[0])
print(pearsonr(testcase_sem.osg, testcase_sem.bert_similarity)[0])
print(pearsonr(testcase_sem.osg, testcase_sem.tasa_similarity)[0])

print(spearmanr(testcase_sem.tasa_similarity, testcase_sem.bert_similarity)[0])
print(spearmanr(testcase_sem.osg, testcase_sem.bert_similarity)[0])
print(spearmanr(testcase_sem.osg, testcase_sem.tasa_similarity)[0])

print(1 - cosine(testcase_sem.tasa_similarity, testcase_sem.bert_similarity))
print(1 - cosine(testcase_sem.osg, testcase_sem.bert_similarity))
print(1 - cosine(testcase_sem.osg, testcase_sem.tasa_similarity))

### Make plotting data file

In [ ]:
pdf = testcase_sem.melt(
    id_vars=['cue', 'target', 'rosg'],
    value_vars=['tasa_similarity', 'bert_similarity'],
    var_name='embedding',
    value_name='similarity'
)

pdf

## Plotting

In [ ]:
testcase_sem.columns

In [ ]:
import altair as alt
from random import sample
alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

samp_df = testcase_sem.sample(n=300, axis=0, random_state=9999)

brush = alt.selection(type='interval')
base = alt.Chart(samp_df).add_selection(brush)

# Configure the points
points_tasa = base.mark_point().encode(
    x=alt.X('rosg:Q', title='', scale=alt.Scale(domain=(0, 1))),
    y=alt.Y('tasa_similarity:Q', title='', scale=alt.Scale(domain=(-1, 1))),
    color=alt.condition(brush, alt.value('blue'), alt.value('grey')),
    tooltip=[
        'cue:N', 'target:N', 'rosg:Q', 'tasa_similarity:Q', 'bert_similarity:Q'
    ]
)

points_bert = base.mark_point().encode(
    x=alt.X('rosg:Q', title='', scale=alt.Scale(domain=(0, 1))),
    y=alt.Y('bert_similarity:Q', title='', scale=alt.Scale(domain=(-1, 1))),
    color=alt.condition(brush, alt.value('orange'), alt.value('grey')),
    tooltip=[
        'cue:N', 'target:N', 'rosg:Q', 'tasa_similarity:Q', 'bert_similarity:Q'
    ]
)

# Configure the ticks
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

x_ticks = base.mark_tick().encode(
    x=alt.X('rosg', axis=tick_axis, scale=alt.Scale(domain=(0, 1))),
    color=alt.condition(brush, alt.value('black'), alt.value('lightgrey'))
)

y_ticks_tasa = base.mark_tick().encode(
    y=alt.Y(
        'tasa_similarity:Q', axis=tick_axis, scale=alt.Scale(domain=(-1, 1))
    ),
    color=alt.condition(brush, alt.value('black'), alt.value('lightgrey'))
)

y_ticks_bert = base.mark_tick().encode(
    y=alt.Y(
        'bert_similarity:Q', axis=tick_axis, scale=alt.Scale(domain=(-1, 1))
    ),
    color=alt.condition(brush, alt.value('black'), alt.value('lightgrey'))
)

left = y_ticks_tasa | (points_tasa & x_ticks)

right = y_ticks_bert | (points_bert & x_ticks)

left | right

In [ ]:
chart = left | right
chart.save('semantic_testcase.html')